Dealing with this horrendous input

In [ ]:
import time
from collections import deque

with open('example.txt', 'r') as f:
    data = f.read().splitlines()

seeds, data = [int(x) for x in data[0].split(':')[1].strip().split(' ')], [x for x in data[2:] if x != '']
print("Seeds: ", seeds)

maps = ['seed-to-soil map:', 'soil-to-fertilizer map:', 'fertilizer-to-water map:', 'water-to-light map:', 'light-to-temperature map:', 'temperature-to-humidity map:', 'humidity-to-location map:']
maps_data = {}

for i, map_name in enumerate(maps):
    start_index = data.index(map_name) + 1
    end_index = data.index(maps[i + 1]) if i + 1 < len(maps) else None
    maps_data[map_name] = [list(map(int, line.split(' '))) for line in data[start_index:end_index]]

print("Maps:")
for map_name, map_data in maps_data.items():
    print(map_name, map_data)


## Part 1

In [ ]:
start = time.time()

final_seeds = []
for s in seeds:
    for mapa in maps_data.keys():
        for group in maps_data[mapa]:
            # group: [destination, source, size]
            # if s in in [source, source + size - 1] or s in [destination, destination + size - 1]:
            if group[1] <= s <= group[1] + group[2] - 1:
                s = group[0] + (s - group[1])
                break
    final_seeds.append(s)

end = time.time()
#print time in ms
print((end - start) * 1000)
    
print(sorted(final_seeds)[0])

    

## Part 2

In [2]:
start = time.time()

# group every 2 values of seeds in tuples
seeds_range = [(seeds[i], seeds[i]+seeds[i + 1]-1) for i in range(0, len(seeds), 2)]
print("Seeds ranges: ", seeds_range)

seeds_range = deque(seeds_range)
print("Seed: ", seeds_range)

def process(range_seeds, range_group):
    # range_seeds: (start, end), range_group: [destination, source, size]
    # mapeia as seeds tendo em conta o grupo, devolvendo o tuplo com o range final das seeds (start, range)
    return (range_group[0], range_seeds[1]-range_seeds[0]+1)

for mapa in maps_data.keys():
    # criar uma cópia por mapa
    unprocessed = deque(seeds_range)
    
    while unprocessed:
        s_range = unprocessed.popleft()
        # para cada mapa, transpor as seeds para o próximo
        for group in sorted(maps_data[mapa], key=lambda x: x[1]):
            # group: [destination, source, size]
            # a usar os grupos sorted para ser não ter de ver TODOS os casos
            
            # se s_range[0] > group[1] + group[2] podemos ignorar o grupo porque não intersectam
            #if s_range[0] > group[1] + group[2] - 1:
            #    continue
            # isto não tem de ser incluído porque se não intersectam com nenhum grupo,
            # o intervalo vai ficar igual, é só adicionar no fim
            
            # se s_range[0] >= group[1] e s_range[1] <= group[1] + group[2] - 1, está contido, processar
            if s_range[0] >= group[1] and s_range[1] <= group[1] + group[2] - 1:
                seeds_range.append(process(s_range, group))

            #se s_range[0] < group[1] e s_range[1] <= group[1] + group[2] - 1, intersecta esquerda, processar
            elif s_range[0] < group[1] and s_range[1] <= group[1] + group[2] - 1:
                unprocessed.append((s_range[0], group[1] - 1))
                seeds_range.append(process((group[1], s_range[1]), group))
                
            #se s_range[0] >= group[1] e s_range[1] > group[1] + group[2] - 1, intersecta direita, processar
            elif s_range[0] >= group[1] and s_range[1] > group[1] + group[2] - 1:
                unprocessed.append((group[1] + group[2], s_range[1]))
                seeds_range.append(process((s_range[0], group[1] + group[2] - 1), group))
                
            # se s_range[0] < group[1] e s_range[1] > group[1] + group[2] - 1, intersecta ambos os lados, processar
            elif s_range[0] < group[1] and s_range[1] > group[1] + group[2] - 1:
                unprocessed.append((s_range[0], group[1] - 1))
                unprocessed.append((group[1] + group[2], s_range[1]))
                seeds_range.append(process((group[1], group[1] + group[2] - 1), group))
            
            # se s_range[0] > group[1] + group[2] - 1, não intersecta, adicionar ao fim
            else:
                seeds_range.append(s_range)
                
print("Seed: ", seeds_range)  


            
end = time.time()
#print time in ms
print((end - start) * 1000)

Seeds ranges:  [(79, 92), (55, 67)]
Seed:  deque([(79, 92), (55, 67)])


KeyboardInterrupt: 